# Air Quality: Design (wrap up) and Implement your Product

Welcome to the final lab of this project. Here again, you'll be working with the dataset you've now become familiar with from the air quality monitoring network in Bogotá [RMCAB](http://201.245.192.252:81/home/map). In this notebook, you will complete the following steps:

1. Import Python packages
2. Load the dataset with missing values filled in (output from the last lab)
3. Use the nearest neighbor method to make a map of PM2.5 in Bogotá
4. Test different values of k for the nearest neighbor method
5. Use the best value of k to make a map of PM2.5 in Bogotá
6. Construct a map animation of PM2.5 in Bogotá
7. Display your map animation

## 1. Import Python Packages

Run the next cell to import the Python packages you'll need for this lab.

Note the `import utils` line. This line imports the functions that were specifically written for this lab. If you want to look at what these functions are, go to `File -> Open...` and open the `utils.py` file to have a look.

In [2]:
# Import Python packages.
import folium # package for animations
import folium.plugins as plugins # extras for animations
import pandas as pd # package for reading in and manipulating data
from sklearn.neighbors import KNeighborsRegressor # package for doing KNN
from datetime import datetime # package for manipulating dates

import utils1 # utility functions defined for this lab

print("All packages imported successfully!")

All packages imported successfully!


## 2. Load the dataset with missing values filled in (output from the last lab)

Run the next cell to read in the dataset that was the final output from the last lab, namely, a dataset with all missing values for the pollutants filled in. 

In [3]:
# Load the dataset with missing values filled in.
full_dataset = pd.read_csv('data/full_data_with_imputed_values.csv')
full_dataset['DateTime'] = pd.to_datetime(full_dataset['DateTime'], dayfirst=True)

full_dataset.head(5)

FileNotFoundError: [Errno 2] No such file or directory: 'data/full_data_with_imputed_values.csv'

## 3. Use the nearest neighbor method to make a map of PM2.5 in Bogotá
Here you use the nearest neighbor method to estimate the values of pollutants at the points between the stations, so you can create a nice visual map of pollution.

In [ ]:
# Define a value for k
k = 3
# Define the target pollutant
target = 'PM2.5'
# Define a grid cell size (higher value implies a finer grid)
n_points_grid = 64
neighbors_model = KNeighborsRegressor(n_neighbors=k, weights = 'distance', metric='sqeuclidean')
# Isolate a single time step from the dataset
time_step = datetime.fromisoformat('2021-04-05T08:00:00')
time_step_data = full_dataset[full_dataset['DateTime'] == time_step]
neighbors_model.fit(time_step_data[['Latitude', 'Longitude']], time_step_data[[target]])
# Generate a map of predictions for Bogotá
predictions_xy, dlat, dlon = utils.predict_on_bogota(neighbors_model, n_points_grid)
utils.create_heat_map(predictions_xy, time_step_data, dlat, dlon, target)

## 4. Test different values of k for the nearest neighbor method
Run the cells below to first calculate the mean absolute error (MAE) for k=1, or in other words, the error associated with using just one nearest neighbor as you did to create the map above. After that, you'll run the same calculation for different values of k. 

The way you're doing this similar to what you did in the previous lab, where you calculated the MAE for using nearby sensor station measurements to estimate the value of PM2.5 at any given sensor station location. Here you'll evaluate the method shown in the map above at each sensor station location as if that station's measurement was replaced with a value from the nearest neighbor station, and then a weighted average of k nearest neighbor stations.


The calculation for mean absolute error that's being performed by the code before is the following:

$$MAE = \frac{1}{n} \sum_{i=1}^{n}{|\rm{actual}_i - \rm{model}_i|}$$
    
Where "n" is the number of samples in the test dataset

In [ ]:
# Make an estimate of mean absolute error for k=1
utils.calculate_mae_for_k(full_dataset, k=1, target_pollutant=target)

After testing k=1, run the following cell to test a range of values for k.

In [ ]:
# Make an estimate of mean absolute error (MAE) for a range of k values.
kmin = 1
kmax = 7

for kneighbors in range(kmin, kmax+1):
    mae = utils.calculate_mae_for_k(full_dataset, k=kneighbors, target_pollutant=target)
    print(f'k = {kneighbors}, MAE = {mae}')

## 5. Use the best value of k to make a map of PM2.5 in Bogotá

Run the cell below to generate a map of PM2.5 values. The map will show the concentration of the chosen pollutant over the city on the selected `end_date`. By clicking on the circles on the map (stations), pop-up plots appear, showing the concentration of the pollutant over the selected time range (from `start_date` to `end_date`) You can change the values of dates and times or `k` to see how the data differs at various times and how the result changes depending on `k`.

In [ ]:
k = 3
start_date = datetime.fromisoformat('2021-08-02T08:00:00')
end_date = datetime.fromisoformat('2021-08-05T08:00:00')

utils.create_heat_map_with_date_range(full_dataset, start_date, end_date, k, target)

## 6. Construct a map animation of PM2.5 in Bogotá
Run the next cell to generate an animation of PM2.5 over a specific time range. You can change k to use a different number of neighbors and change the dates and times to look at a different time range. 

In [ ]:
# Choose parameters for the animation
k = 3
n_points_grid = 64
# Filter a date range
start_date = datetime.fromisoformat('2021-08-04T08:00:00')
end_date = datetime.fromisoformat('2021-08-05T08:00:00')

# Create the features for the animation (these are the shapes that will appear on the map)
features = utils.create_animation_features(full_dataset, start_date, end_date, k, n_points_grid, target)
print('Features for the animation created successfully! Run the next cell to see the result!')

## 7. Display your map animation

Run the next cell to display the animation you created!

In [ ]:
# Create the map animation using the folium library
map_animation = folium.Map(location=[4.7110, -74.0721], zoom_start = 11) 
# Add the features to the animation
plugins.TimestampedGeoJson(
    {"type": "FeatureCollection", "features": features},
    period="PT1H",
    duration='PT1H',
    add_last_point=True
).add_to(map_animation)

# Run the animation
map_animation

## **Congratulations on finishing this lab!**

**Keep up the good work :)**